In [1]:
import gradio as gr
import ollama

In [2]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [21]:
import time
import gradio as gr

def slow_echo(message, history):
    history.append({"role" : "system", "content" : system_message})
    history.append({"role":"user", "content":message})
    response = ollama.chat(
        model = 'qwen2.5:1.5b',
        messages= history,
        stream=True
    )
    result = ""
    for word in response:
        result += word.message.content
        yield result
    
demo = gr.ChatInterface(
    slow_echo,
    type="messages",
    flagging_mode="manual",
    flagging_options=["Like", "Spam", "Inappropriate", "Other"],
    save_history=True,
)

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.<br>
With tools, you can write a function, and have the LLM call that function as part of its response.<br>
Sounds almost spooky.. we're giving it the power to run code on our machine?<br>
Well, kinda.

In [16]:
# Let's start by making a useful function



def get_ticket_price(destination_city):
    '''Get the price of a return ticket to the destination city. 
    Call this whenever you need to know the ticket price, 
    for example when a customer asks 'How much is a ticket to this city'
    '''
    ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [17]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [18]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

In [22]:
def slow_echo(message, history):
    # print(history)
    history.append({"role" : "system", "content" : system_message})
    history.append({"role":"user", "content":message})
    response = ollama.chat(
        model = "qwen2.5:1.5b",
        messages=history,
        stream=True,
        tools=tools
    )
    result = ""
    for word in response:
        result += word.message.content
        yield result

demo = gr.ChatInterface(
    slow_echo,
    type="messages",
    flagging_mode="manual",
    flagging_options=["Like", "Spam", "Inappropriate", "Other"],
    save_history=True,
)

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.
